# Setup

### Import libraries

In [1]:
import os

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import bottleneck as bn

### Set seed

In [2]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Data

### Load file

In [3]:
path = '/data/ephemeral/home/data/cold/'
save_dir = '../saved'
model_path = save_dir + 'ncf.pth'
mode = 5

full_df = pd.read_csv(path + 'full.csv')
train_df = pd.read_csv(path + f'{mode}shot/train_{mode}.csv')

val_k = pd.read_csv(path + f'{mode}shot/val_{mode}_k.csv')
test_k = pd.read_csv(path + f'{mode}shot/test_{mode}_k.csv')

val_n = pd.read_csv(path + f'{mode}shot/val_{mode}_n.csv')
test_n = pd.read_csv(path + f'{mode}shot/test_{mode}_n.csv')

### Reindex dataframe

In [4]:
users = full_df['user_id'].unique()
items = full_df['item_id'].unique()

user2idx = {user: i for i, user in enumerate(users)}
item2idx = {item: i for i, item in enumerate(items)}

In [5]:
# 리인덱싱 수행
train_df['user_id'] = train_df['user_id'].map(user2idx)
train_df['item_id'] = train_df['item_id'].map(item2idx)

val_k['user_id'] = val_k['user_id'].map(user2idx)
val_k['item_id'] = val_k['item_id'].map(item2idx)

test_k['user_id'] = test_k['user_id'].map(user2idx)
test_k['item_id'] = test_k['item_id'].map(item2idx)

val_n['user_id'] = val_n['user_id'].map(user2idx)
val_n['item_id'] = val_n['item_id'].map(item2idx)

test_n['user_id'] = test_n['user_id'].map(user2idx)
test_n['item_id'] = test_n['item_id'].map(item2idx)

In [6]:
n_users = len(users)
n_items = len(items)

del full_df

In [7]:
val_users = val_k['user_id'].unique()
test_users = test_k['user_id'].unique()

In [8]:
# {user_id: [item_ids]} 딕셔너리 생성, train 에서는 neg_sampling에서 마스킹으로, Top K 추천에서는 scores에 대한 마스킹으로 사용 됌
train_user_train_items = {k: list(v['item_id'].values) for k, v in train_df.groupby('user_id')}
val_user_train_items = {k: list(v['item_id'].values) for k, v in val_k.groupby('user_id')}
test_user_train_items = {k: list(v['item_id'].values) for k, v in test_k.groupby('user_id')}

# val, test 유저들에 대한 정답 아이템 리스트 딕셔너리
val_actual = {k: list(v['item_id'].values) for k, v in val_n.groupby('user_id')}
test_actual = {k: list(v['item_id'].values) for k, v in test_n.groupby('user_id')}

In [9]:
del val_k, test_k, val_n, test_n

### Create COO

In [10]:
def create_coo_matrix(df, n_users, n_items):
    # torch 텐서로 변환
    user_id_tensor = torch.tensor(df['user_id'].values, dtype=torch.long)
    item_id_tensor = torch.tensor(df['item_id'].values, dtype=torch.long)
    label_tensor = torch.ones(len(df), dtype=torch.float32)

    # COO 희소 텐서 생성
    indices = torch.stack([user_id_tensor, item_id_tensor])
    values = label_tensor
    size = (n_users, n_items)  # 전체 유저 x 전체 아이템 크기로 지정

    return torch.sparse_coo_tensor(indices, values, size)

In [11]:
train_coo = create_coo_matrix(train_df, n_users, n_items)

### Dataset Class

In [13]:
class BPRDataset(Dataset):
    '''
    베이지안 개인화 순위(BPR)를 위한 PyTorch 데이터셋.

    args:
        coo (torch.sparse.coo_tensor): COO 형식의 사용자-아이템 상호작용 행렬.
        user_train_items (dict): 각 사용자가 상호작용한 아이템 목록을 매핑하는 딕셔너리.
        n_negs (int): 긍정 샘플당 생성할 부정 샘플의 수. 기본값은 1.
    '''
    def __init__(self, coo, user_train_items, n_negs=1):
        self.users, self.pos_items = coo._indices()
        self.user_train_items = user_train_items
        self.n_negs = n_negs
        self.n_items = coo.shape[1]
        self.n_inter = coo._values().shape[0]

    def __len__(self):
        return self.n_inter

    def __getitem__(self, idx):
        user = self.users[idx]
        pos_item = self.pos_items[idx]
        neg_item = self._get_neg_item(user.item())
        return user, pos_item, neg_item
    
    # def _get_neg_items(self, user):
    #     neg_items = set()
    #     while len(neg_items) < self.n_negs:
    #         neg_item = torch.randint(0, self.n_items, (1,)).item()
    #         if neg_item not in self.user_pos_items[user]:
    #             neg_items.add(neg_item)
    #     return torch.LongTensor(list(neg_items))
    
    def _get_neg_item(self, user):
        train_items = set(self.user_train_items[user])
        neg_item = torch.randint(0, self.n_items, (1,)).item()
        while neg_item in train_items:
            neg_item = torch.randint(0, self.n_items, (1,)).item()
        return neg_item


### DataLoader

In [14]:
train_dataset = BPRDataset(train_coo, train_user_train_items)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)

In [16]:
del train_df

## Model

In [17]:
class NCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim=64, dropout=0.2):
        super(NCF, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.dropout = dropout

        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        self.mlp = nn.Sequential(  # [batch_size, emb_dim * 2]
            nn.Dropout(dropout),
            nn.Linear(emb_dim * 2, emb_dim),  # [batch_size, emb_dim]
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(emb_dim, emb_dim // 2),  # [batch_size, emb_dim // 2]
            nn.ReLU(),
            nn.Linear(emb_dim // 2, 1)  # [batch_size, 1]
        )

        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_normal_(self.user_emb.weight)
        nn.init.xavier_normal_(self.item_emb.weight)
        for layer in self.mlp:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)
                nn.init.constant_(layer.bias, 0.0)

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        concat = torch.cat([user_emb, item_emb], dim=-1)  # [batch_size, emb_dim * 2]
        return self.mlp(concat).squeeze()  # [batch_size]

### BPR loss

In [18]:
# BPR 손실 함수
class BPRLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pos_scores, neg_scores):
        return -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()

In [19]:
def recommend_items(model, user_id, num_items, top_k=10, user_train_items=None):
    """
    단일 유저에 대해 전체 아이템에 대한 스코어를 계산한 후,
    이미 학습에 활용된 아이템(train_items)이 있을 경우 이를 마스킹(-무한대로 대체)하고,
    bn.argpartition을 활용해 상위 top_k 아이템을 효율적으로 추출하는 함수.
    
    args:
        model: user_id와 item_id의 텐서를 입력받아 스코어를 반환하는 추천 모델.
        user_id (int): 추천을 위한 대상 유저 ID.
        num_items (int): 전체 아이템의 개수.
        top_k (int): 추천할 아이템 수.
        train_items (list 또는 np.array, optional): 학습 시 활용된 해당 유저의 아이템 인덱스 리스트.
    
    return:
        추천 아이템 인덱스 리스트 (정렬되어 있음)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # user_id를 전체 아이템 개수만큼 반복하여 텐서 생성
    user_ids = torch.full((num_items,), user_id, dtype=torch.long).to(device)
    # 모든 아이템의 인덱스 생성
    item_ids = torch.arange(num_items, dtype=torch.long).to(device)
    
    with torch.no_grad():
        scores = model(user_ids, item_ids)
    
    # train_items가 제공되면 해당 아이템의 스코어를 마스킹 처리 (-무한대값으로 대체)
    if user_train_items is not None:
        # torch indexing은 list나 array로도 발동됨
        scores[user_train_items] = -float('inf')
    
    # GPU에 있을 경우 CPU로 옮긴 후 numpy 배열로 변환
    scores_np = scores.cpu().numpy()
    
    # bottleneck의 argpartition을 사용하여 상위 top_k의 후보 인덱스를 추출
    # 음수 부호를 취해 내림차순 정렬 효과를 냄.
    candidate_indices = bn.argpartition(-scores_np, top_k-1)[:top_k]
    
    # argpartition은 정렬되어 있지 않으므로, 위 후보들에 대해 추가 정렬(내림차순) 수행
    sorted_top_indices = candidate_indices[np.argsort(-scores_np[candidate_indices])]
    
    return sorted_top_indices.tolist()

### Metrics

In [20]:
def recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = set(predicted[:k])
    return len(actual_set & predicted_at_k) / k

def precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = set(predicted[:k])
    return len(actual_set & predicted_at_k) / len(predicted_at_k)

def ndcg_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    dcg = sum([1 / np.log2(i + 2) for i, p in enumerate(predicted_at_k) if p in actual_set])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(len(actual_set), k))])
    return dcg / idcg if idcg > 0 else 0.0

def map_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    score = 0.0
    hit_count = 0.0
    for i, item in enumerate(predicted_at_k):
        if item in actual_set:
            hit_count += 1.0
            score += hit_count / (i + 1)
    return score / len(actual_set) if actual else 0.0


In [21]:
def evaluate(model, user_list, n_items, top_k, user_train_items, user_actual):
    """
    모델의 평가를 위해 전체 아이템에 대한 추천 리스트를 생성하고
    recall@k 등의 평가 지표를 계산하는 함수.
    
    args:
      model (nn.Module): 추천 스코어를 계산하는 모델.
      user_list (iterable): 평가할 유저 ID 리스트.
      n_items (int): 전체 아이템의 개수.
      top_k (int): 상위 추천 아이템 수 (예: 10).
      user_train_items (dict): 각 유저마다 학습에 사용된 아이템 인덱스 리스트 – 평가 시 해당 아이템은 마스킹 처리함.
      user_actual (dict): 각 유저의 실제 정답(ground truth) 아이템 리스트.
    
    return:
      전체 유저에 대한 평균 recall@k 값.
    """
    model.eval()  # 평가 모드로 전환 (dropout, batchnorm 등 고정)
    recalls, precisions, ndcgs, maps = [], [], [], []
    
    with torch.no_grad():
        for user in tqdm(user_list, desc="Evaluating", leave=False):
            # 기존에 작성한 recommend_items 함수 사용 (학습에 사용한 아이템은 마스킹)
            recommendations = recommend_items(model, user, n_items, top_k, user_train_items[user])
            # 추천 결과와 실제 정답(dict나 list)에 따라 메트릭 계산
            recalls.append(recall_at_k(user_actual[user], recommendations, top_k))
            precisions.append(precision_at_k(user_actual[user], recommendations, top_k))
            ndcgs.append(ndcg_at_k(user_actual[user], recommendations, top_k))
            maps.append(map_at_k(user_actual[user], recommendations, top_k))

    model.train()  # 평가 후 다시 학습 모드로 전환

    metrics = {
        'recall': np.mean(recalls),
        'precision': np.mean(precisions),
        'ndcg': np.mean(ndcgs),
        'map': np.mean(maps),
    }

    return metrics


### 학습 파라미터 설정

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
emb_dim = 64
dropout = 0.2
model = NCF(n_users, n_items, emb_dim, dropout)
model = model.to(device)
criterion = BPRLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## 학습

In [24]:
epochs = 10
valid_interval = 1
best_val_ndcg = -1
best_val_metrics = None

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for user, pos_item, neg_item in tqdm(train_loader, desc=f'Training Epoch {epoch+1}'):
        user = user.to(device)
        pos_item = pos_item.to(device)
        neg_item = neg_item.to(device)

        pos_scores = model(user, pos_item)
        neg_scores = model(user, neg_item)

        loss = criterion(pos_scores, neg_scores)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)

 # 평가: val_users, n_items, top_k, 그리고 유저별 학습/정답 아이템 정보 사용
    if (epoch + 1) % valid_interval == 0:
        # Evaluate on validation users
        val_metrics = evaluate(model, val_users, n_items, 10, val_user_train_items, val_actual)

        print(f'Epoch {epoch+1}, Loss: {avg_train_loss:.4f} | '
              f'Recall@10: {val_metrics['recall']:.4f}, Precision@10: {val_metrics['precision']:.4f}, '
              f'NDCG@10: {val_metrics['ndcg']:.4f}, MAP@10: {val_metrics['map']:.4f}')

        # Check if this is the best validation NDCG
        if val_metrics['ndcg'] > best_val_ndcg:
            best_val_ndcg = val_metrics['ndcg']
            best_val_metrics = val_metrics
            best_epoch = epoch+1
            torch.save(model.state_dict(), model_path)
            print(f'Model saved to {model_path}')
    else:
        print(f'Epoch {epoch+1}, Loss: {avg_train_loss:.4f}')
    
# Evaluate on test users
model.load_state_dict(torch.load('../saved/ncf.pth', weights_only=True))
test_metrics = evaluate(model, test_users, n_items, 10, test_user_train_items, test_actual)

# Print best validation metrics
if best_val_metrics:
    print(f'Best Epoch:{best_epoch}, Val Metrics - Recall@10: {best_val_metrics['recall']:.4f}, '
          f'Precision@10: {best_val_metrics['precision']:.4f}, '
          f'NDCG@10: {best_val_metrics['ndcg']:.4f}, MAP@10: {best_val_metrics['map']:.4f}')
# Print test metrics
print(f'Test Metrics - Recall@10: {test_metrics['recall']:.4f}, '
      f'Precision@10: {test_metrics['precision']:.4f}, '
      f'NDCG@10: {test_metrics['ndcg']:.4f}, MAP@10: {test_metrics['map']:.4f}')

Training Epoch 1: 100%|██████████| 104047/104047 [10:16<00:00, 168.87it/s]


Epoch 1, Loss: 0.0875 | Recall@10: 0.0551, Precision@10: 0.0551, NDCG@10: 0.0992, MAP@10: 0.0472


Training Epoch 2: 100%|██████████| 104047/104047 [10:28<00:00, 165.57it/s]


Epoch 2, Loss: 0.0800 | Recall@10: 0.0765, Precision@10: 0.0765, NDCG@10: 0.1173, MAP@10: 0.0564


Training Epoch 3: 100%|██████████| 104047/104047 [10:25<00:00, 166.29it/s]


...

Training Epoch 9: 100%|██████████| 104047/104047 [10:23<00:00, 166.95it/s]


Epoch 9, Loss: 0.0656 | Recall@10: 0.0718, Precision@10: 0.0718, NDCG@10: 0.1187, MAP@10: 0.0585


Training Epoch 10: 100%|██████████| 104047/104047 [10:31<00:00, 164.79it/s]


Epoch 10, Loss: 0.0652 | Recall@10: 0.0734, Precision@10: 0.0734, NDCG@10: 0.1242, MAP@10: 0.0610


Best Validation Metrics - Recall@10: 0.0901, Precision@10: 0.0901, NDCG@10: 0.1456, MAP@10: 0.0745
Test Metrics - Recall@10: 0.0741, Precision@10: 0.0741, NDCG@10: 0.1267, MAP@10: 0.0624


### Train 안할 시 모델 로드

In [ ]:
# model.load_state_dict(torch.load('../saved/ncf.pth', weights_only=True))
# model.eval()

### 테스트 유저에게 추천 및 결과 저장

In [39]:
# 빈 리스트를 생성하여 추천 결과를 저장
recommendations = []

# 각 사용자에 대해 추천 아이템을 생성하고 리스트에 추가
for user_id in tqdm(test_users):
    recommended_items = recommend_items(model, user_id, n_items, 10, test_user_train_items[user_id])
    for item_id in recommended_items:
        recommendations.append({'user_id': user_id, 'item_id': item_id})

# 리스트를 데이터프레임으로 변환
result = pd.DataFrame(recommendations)

100%|██████████| 13754/13754 [00:15<00:00, 885.58it/s]


In [40]:
idx2user = {v: k for k, v in user2idx.items()}
idx2item = {v: k for k, v in item2idx.items()}

result['user_id'] = result['user_id'].map(idx2user)
result['item_id'] = result['item_id'].map(idx2item)

result.to_csv('../saved/result.csv', index=False)

### 딕셔너리로 추천 결과 생성하는 코드

In [25]:
# test_recommendations = {user_id: recommend_items(model, user_id, n_items, 10, test_user_train_items[user_id]) for user_id in tqdm(test_users)}

100%|██████████| 13754/13754 [00:15<00:00, 894.46it/s]


In [31]:
# result = pd.DataFrame([
#     {'user_id': user, 'item_id': item}
#     for user, items in test_recommendations.items()
#     for item in items
# ])